In [32]:
import requests
from bs4 import BeautifulSoup
from sympy import per
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import time 
from time import sleep

stocks = ['XENE', 'NEX', 'OXY', 'KR', 'XOM', 'WMT', 'JNJ', 'KO', 'GOOGL', 'MA', 'AAPL', 'DDOG', 'PBF', 'AMZN', 'TSLA',
         'PEP', 'MSFT', 'MCD', 'NVDA', 'GS', 'JPM', 'SQ', 'PLTR', 'ROKU', 'TDOC']

PFA, v, score, stockslist, industry, market_cap, short_float, price, beta, dividend = [], [], [], [], [], [], [], [], [], []

for stock in stocks:
    ticker = yf.Ticker(stock)
    
    #Current Price
    try:
        current_price = ticker.info['currentPrice']
    except:
        current_price = 0
        
    #52-Week High
    try:
        ATH = ticker.info['fiftyTwoWeekHigh']
    except:
        ATH = 1
    percent_from_52 = round((1 - current_price/ATH)*100, 2)
    
    if percent_from_52 == 100:
        PFA.append(0)
    else:
         PFA.append(str(percent_from_52) + '%')
    
    #Volatility
    url = f'https://www.alphaquery.com/stock/{stock}/volatility-option-statistics/180-day/historical-volatility'
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    
    try:
        volatility = soup.findAll('div', class_ = "indicator-figure-inner")[0]
        vol = float(volatility.text)*100
        vol = round(vol, 2)
        if vol == 0:
            v.append("No data")
        else:
            v.append(str(round(vol, 1)) + '%')
    except:
        vol = "No Data"
        v.append(vol)
    
    #Market Cap
    try:
        mkcp = ticker.info['marketCap']
        market_cap.append(round(mkcp/1000000000))
    except:
        mkcp = 0
        market_cap.append("No data")
 
    #Industry
    try:
        industry.append(ticker.info['sector'])
    except:
        industry.append("No data")
    
    #Short Float
    try:
        shorts = ticker.info['shortPercentOfFloat']
        shorts = round(float(shorts*100), 2)
        shortss = str(shorts) + '%'
        short_float.append(shortss)
    except:
        shorts = 0
        short_float.append("No data")
    
    #Strength Score
    if vol == 0:
        score.append("N/A")
    else:
        strength_score = ((0-percent_from_52 + (vol/1.5))/2 + (shorts/5))/2
        strength_score += (div/10)
        score.append(round(strength_score, 1))
        
    stockslist.append(stock)    
    price.append('$' + str(round(current_price, 2)))
        
data = {'Stock': stockslist,
        'Industry': industry,
        'Current Price': price,
        'Market Cap (B)': market_cap,
        '% Float Short': short_float,
        '% From 52w High': PFA,
        'Volatility': v,
        'Strength Score': score}

df = pd.DataFrame(data)
df = df.sort_values(by='Strength Score', ascending=False)
df.reset_index(drop = True, inplace=True)
df.index = np.arange(1, len(df)+1)

df

#Date: September 15, 2022

,Stock,Industry,Current Price,Market Cap (B),% Float Short,% From 52w High,Volatility,Strength Score
1,XENE,Healthcare,$38.56,2,3.84%,6.84%,62.4%,9.1
2,OXY,Energy,$65.04,61,13.8%,15.67%,54.4%,6.5
3,NEX,Energy,$8.91,2,3.27%,28.7%,71.5%,5.1
4,PBF,Energy,$28.73,3,17.81%,34.88%,70.5%,4.8
5,TSLA,Consumer Cyclical,$302.65,304,2.37%,26.98%,65.0%,4.3
6,XOM,Energy,$94.32,393,0.96%,10.66%,36.8%,3.6
7,AAPL,Technology,$151.49,2485,0.65%,17.19%,35.5%,1.7
8,MCD,Consumer Cyclical,$253.08,189,0.83%,6.66%,19.7%,1.7
9,PEP,Consumer Defensive,$165.54,229,0.79%,8.57%,20.4%,1.3
10,WMT,Consumer Defensive,$133.08,361,1.06%,17.22%,32.0%,1.1
